## Prediction of district's median housing price

### Fetching the data

In [1]:
import os, io
import tarfile
import requests
import hashlib

In [2]:
DOWNLOAD_ROOT = "https://raw.githubusercontent.com/ageron/handson-ml/master/"
HOUSING_PATH = os.path.join("datasets", "housing")
HOUSING_URL = DOWNLOAD_ROOT+"datasets/housing/housing.tgz"

def fetch_housing_data(housing_url=HOUSING_URL, housing_path=HOUSING_PATH):
    if not os.path.isdir(housing_path):
        os.makedirs(housing_path)
    tgz_path = os.path.join(housing_path, "housing.tgz")
    response = requests.get(housing_url)
    with open(tgz_path, 'wb') as f:
        f.write(response.content)
    housing_tgz = tarfile.open(tgz_path)
    housing_tgz.extractall(path=housing_path)
    housing_tgz.close()    

In [ ]:
#fetch_housing_data()

In [ ]:
os.listdir(HOUSING_PATH)

In [7]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedShuffleSplit

In [4]:
def load_housing_data(housing_path=HOUSING_PATH):
    csv_path = os.path.join(housing_path, 'housing.csv')
    return pd.read_csv(csv_path)

In [ ]:
housing = load_housing_data()
housing.head()

In [ ]:
housing.info()

In [ ]:
housing.isnull().sum().sort_values(ascending=False)

In [ ]:
housing['ocean_proximity'].value_counts()

In [ ]:
housing.describe()

In [ ]:
sns.set()
sns.set_style("whitegrid")
#sns.plotting_context()
fig, ax = plt.subplots(figsize=(20,20))
sns.set_context("notebook", rc={"font.size":20,"axes.titlesize":20,"axes.labelsize":20,"xtick.labelsize":20,"ytick.labelsize":20})
plt.subplot(331) # rows, ncols, fignum
sns.distplot(housing['housing_median_age'], kde=False)
plt.subplot(332)
sns.distplot(housing['total_rooms'], kde=False)
plt.subplot(333)
#sns.distplot(housing['total_bedrooms'])
plt.subplot(334)
sns.distplot(housing['population'], kde=False)
plt.subplot(335)
sns.distplot(housing['households'], kde=False)
plt.subplot(336)
sns.distplot(housing['median_income'], kde=False)
plt.subplot(337)
sns.distplot(housing['median_house_value'], kde=False)
plt.xticks(rotation=90) 
plt.tight_layout()

In [5]:
import numpy as np

In [6]:
def split_train_test(data, test_ratio=0.2):
    np.random.seed(5)
    shuffle_indices = np.random.permutation(len(data))
    test_set_size = int(len(data)*test_ratio)
    test_indices = shuffle_indices[:test_set_size]
    train_indices = shuffle_indices[test_set_size:]
    return data.iloc[train_indices], data.iloc[test_indices]

def test_set_check(identifier, test_ratio, hash):
    return hash(np.int64(identifier)).digest()[-1]<256*test_ratio

def split_train_test_by_id(data, test_ratio, id_column, hash=hashlib.md5):
    ids = data[id_column]
    in_test_set = ids.apply(lambda id_: test_set_check(id_, test_ratio, hash))
    return data.loc[~in_test_set], data.loc[in_test_set]

In [ ]:
train_set, test_set = split_train_test(housing, 0.2)

In [ ]:
display(train_set.shape)
display(test_set.shape)
type(train_set)

### add id column using index

In [ ]:
housing_with_id = housing.reset_index()
housing_with_id.head()

In [ ]:
train_set, test_set = split_train_test_by_id(housing_with_id, 0.2, "index")
display(train_set.shape)
display(test_set.shape)

In [ ]:
housing['income_cat'] = np.ceil(housing["median_income"] / 1.5)
housing['income_cat'].where(housing['income_cat']<5, 5.0, inplace=True)
housing.head()

In [ ]:
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=5)
for train_index, test_index in split.split(housing, housing['income_cat']):
    strat_train_set = housing.loc[train_index]
    strat_test_set = housing.loc[test_index]

In [ ]:
strat_test_set['income_cat'].value_counts()/len(strat_test_set)

In [ ]:
housing['income_cat'].value_counts()/len(housing)

In [ ]:
### remove 'income_cat' so that data is back to normal after split
for set_ in (strat_train_set, strat_test_set):
    set_.drop('income_cat', axis=1, inplace=True)

In [ ]:
display(strat_test_set.head())
display(strat_train_set)

In [ ]:
housing = strat_train_set.drop("median_house_value", axis=1)
housing_labels = strat_train_set["median_house_value"].copy()

In [ ]:
housing.plot(kind='scatter', x="longitude", y="latitude",
             alpha=0.4, s=housing['population']/100, label='population', 
             c="median_house_value", cmap=plt.get_cmap("jet"), colorbar=True, figsize=(10,7))
plt.legend()

In [ ]:
corr_matrix = housing.corr()
corr_matrix

In [ ]:
corr_matrix["median_house_value"].sort_values(ascending=False)

In [ ]:
from pandas.plotting import scatter_matrix

attributes = ["median_house_value", "median_income", "total_rooms",
              "housing_median_age"]
scatter_matrix(housing[attributes], figsize=(15, 15))
#sns.pairplot(housing[attributes], size=6)
#plt.tight_layout()

In [ ]:
sns.jointplot(x='median_income',y='median_house_value', data=housing, alpha=0.1)

In [ ]:
housing['rooms_per_household'] = housing['total_rooms']/housing['households']
housing['bedrooms_per_room'] = housing['total_bedrooms']/housing['total_rooms']
housing['population_per_household'] = housing['population']/housing['households']

In [ ]:
corr_matrix = housing.corr()
corr_matrix['median_house_value'].sort_values(ascending=False)


### Data cleaning

In [ ]:
#housing.dropna(subset=['total_bedrooms']) 
#housing.drop('total_bedrooms', axis=1, inplace=True)
total_bedrooms_median = housing['total_bedrooms'].median()
housing['total_bedrooms'].fillna(total_bedrooms_median, inplace=True)

In [ ]:
housing['total_bedrooms'].value_counts()

In [ ]:
from sklearn.preprocessing import Imputer

In [ ]:
housing_num = housing.drop('ocean_proximity', axis=1)

In [ ]:
imputer = Imputer(strategy='median')
imputer.fit(housing_num)
imputer.statistics_

In [ ]:
housing_num.median()

In [ ]:
X = imputer.transform(housing_num)
housing_tr = pd.DataFrame(X, columns=housing_num.columns)
housing_tr.head()


In [ ]:
housing_cat = housing['ocean_proximity']
display(housing_cat.head(10))
housing_cat_encoded, housing_categories = housing_cat.factorize()
housing_cat_encoded[:10]
housing['ocean_proximity'] = housing_cat_encoded

In [ ]:
housing_categories[:10]

In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
one_hot = OneHotEncoder()
housing_cat_1hot = one_hot.fit_transform(housing_cat_encoded.reshape(-1, 1))
housing_cat_1hot

In [ ]:
housing_cat_1hot.toarray()

### Creating Pipeline

In [8]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder #, CategoricalEncoder
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import Imputer

In [9]:
class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.attribute_names].values        

In [116]:
housing_data = load_housing_data()

In [117]:
housing_num = housing_data.drop('ocean_proximity', axis=1)
housing_cat = housing_data['ocean_proximity']
housing_cat_encoded, housing_categories = housing_cat.factorize() #pd.get_dummies(housing_cat)
housing_data['ocean_proximity'] = housing_cat_encoded


housing_data['income_cat'] = np.ceil(housing_data["median_income"] / 1.5)
housing_data['income_cat'].where(housing_data['income_cat']<5, 5.0, inplace=True)

split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=5)
for train_index, test_index in split.split(housing_data, housing_data['income_cat']):
    strat_train_set = housing_data.loc[train_index]
    strat_test_set = housing_data.loc[test_index]
    
### remove 'income_cat' so that data is back to normal after split
for set_ in (strat_train_set, strat_test_set):
    set_.drop('income_cat', axis=1, inplace=True)
    
housing_train = strat_train_set.drop("median_house_value", axis=1)
housing_train_labels = strat_train_set["median_house_value"].copy()

housing_test = strat_test_set.drop("median_house_value", axis=1)
housing_test_labels = strat_test_set["median_house_value"].copy()


num_attribs = list(housing_train) # list of all columns in df
cat_attribs = ['ocean_proximity']

In [118]:
num_pipeline = Pipeline([
    ('selector', DataFrameSelector(num_attribs)),
    ('imputer', Imputer(strategy="median")),
    ('std_scaler', StandardScaler())
])
cat_pipeline = Pipeline([
    ('selector', DataFrameSelector(cat_attribs)),
#    ('lbl_encoder', LabelEncoder()),
    ('1hot_encoder', OneHotEncoder()) # use CategoricalEncoder when available
])

In [119]:
from sklearn.pipeline import FeatureUnion

In [120]:
full_pipeline = FeatureUnion(transformer_list=[('num_pipeline', num_pipeline),
                                               ('cat_pipeline', cat_pipeline)
                                              ]
                            )

In [121]:
housing_prepared = full_pipeline.fit_transform(housing_train)
housing_prepared

<16512x14 sparse matrix of type '<class 'numpy.float64'>'
	with 165120 stored elements in Compressed Sparse Row format>

In [122]:
display(housing_prepared.toarray().shape)
housing.shape

(16512, 14)

(16512, 9)

In [123]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [124]:
lin_reg = LinearRegression()
lin_reg.fit(housing_prepared, housing_train_labels)
housing_test_prepared = full_pipeline.transform(housing_test)
y_pred = lin_reg.predict(housing_test_prepared)
print(np.sqrt(mean_squared_error(housing_test_labels, y_pred)))
r2_score(housing_test_labels, y_pred)

67447.7274144


0.65625230384905531

In [125]:
from sklearn.tree import DecisionTreeRegressor

In [126]:
tree_reg = DecisionTreeRegressor()
tree_reg.fit(housing_prepared, housing_train_labels)
y_pred = tree_reg.predict(housing_test_prepared)
print(np.sqrt(mean_squared_error(housing_test_labels, y_pred)))
r2_score(housing_test_labels, y_pred)

67514.8691281


0.65556758702200368

In [127]:
from sklearn.model_selection import cross_val_score

In [128]:
scores = cross_val_score(tree_reg, housing_prepared, housing__train_labels, scoring='neg_mean_squared_error', cv=5)
tree_rmse_scores = np.sqrt(-scores)

In [129]:
def display_scores(scores):
    print("Scores", scores)
    print("Mean", scores.mean())
    print("Std Dev", scores.std())

In [130]:
display_scores(tree_rmse_scores)

Scores [ 67662.52005466  67446.00483985  71200.2531181   70255.09568362
  67625.60511713]
Mean 68837.8957627
Std Dev 1573.38519733


In [131]:
from sklearn.ensemble import RandomForestRegressor

In [132]:
f_reg = RandomForestRegressor()
f_reg.fit(housing_prepared, housing__train_labels)
y_pred = f_reg.predict(housing_test_prepared)
print(np.sqrt(mean_squared_error(housing_test_labels, y_pred)))
print(r2_score(housing_test_labels, y_pred))
scores = cross_val_score(f_reg, housing_prepared, housing__train_labels, scoring='neg_mean_squared_error', cv=5)
tree_rmse_scores = np.sqrt(-scores)
display_scores(tree_rmse_scores)

51705.1410001
0.797990121483
Scores [ 51486.53194986  51744.9725849   52462.39455811  50872.77569172
  50057.63184948]
Mean 51324.8613268
Std Dev 813.248385367


In [133]:
from sklearn.model_selection import GridSearchCV

In [134]:
param_grid = [
    {'n_estimators':[3, 10, 30], 'max_features': [2, 4, 6, 8]},
    {'bootstrap':[False], 'n_estimators':[3, 10], 'max_features':[2, 3, 4]}    
]

f_reg = RandomForestRegressor()
grid_search = GridSearchCV(f_reg, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(housing_prepared, housing__train_labels)

GridSearchCV(cv=5, error_score='raise',
       estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid=[{'n_estimators': [3, 10, 30], 'max_features': [2, 4, 6, 8]}, {'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2, 3, 4]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_mean_squared_error', verbose=0)

In [135]:
display(grid_search.best_params_)
display(grid_search.best_estimator_)
display(grid_search.cv_results_)
feature_importances = grid_search.best_estimator_.feature_importances_
attributes = list(num_attribs) + list(housing_categories)
sorted(zip(feature_importances, attributes), reverse=True)

{'max_features': 8, 'n_estimators': 30}

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features=8, max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=30, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False)

/home/suri/.virtualenvs/dl-py3/lib/python3.5/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('mean_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/home/suri/.virtualenvs/dl-py3/lib/python3.5/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/home/suri/.virtualenvs/dl-py3/lib/python3.5/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/home/sur

{'mean_fit_time': array([ 0.37136083,  1.22466235,  3.67439518,  0.53308239,  1.77266188,
         5.28737679,  0.71534271,  2.38833551,  7.14256415,  0.90587072,
         3.00745168,  8.97597623,  0.56907997,  1.88338137,  0.67855692,
         2.26353121,  0.821491  ,  2.73198094]),
 'mean_score_time': array([ 0.00320969,  0.00819597,  0.02212682,  0.00323858,  0.00817456,
         0.02214737,  0.00324497,  0.00819292,  0.02210178,  0.00324802,
         0.00818348,  0.0215107 ,  0.00345488,  0.00894303,  0.00344782,
         0.00887966,  0.00344248,  0.00895572]),
 'mean_test_score': array([ -3.89032876e+09,  -2.98411236e+09,  -2.73266892e+09,
         -3.66557054e+09,  -2.81139779e+09,  -2.55667607e+09,
         -3.48650152e+09,  -2.72773634e+09,  -2.51508430e+09,
         -3.45308557e+09,  -2.70553366e+09,  -2.48920777e+09,
         -3.73493771e+09,  -2.94584884e+09,  -3.58112484e+09,
         -2.75413376e+09,  -3.43264612e+09,  -2.68714392e+09]),
 'mean_train_score': array([ -1.074

[(0.42197708677954288, 'median_income'),
 (0.15028956808435334, 'INLAND'),
 (0.1172799158018596, 'longitude'),
 (0.10420634884124597, 'latitude'),
 (0.048902345869361499, 'housing_median_age'),
 (0.0363903056345493, 'population'),
 (0.03292631207099618, 'ocean_proximity'),
 (0.029373164067270842, 'total_rooms'),
 (0.027143600944322298, 'total_bedrooms'),
 (0.024919775055763014, 'households'),
 (0.0029184553452584945, 'NEAR OCEAN'),
 (0.0025123173584525684, '<1H OCEAN'),
 (0.0011087565410971419, 'NEAR BAY'),
 (5.2047605926778863e-05, 'ISLAND')]

### Import library to save model

In [68]:
from sklearn.externals import joblib

### Save Model

In [69]:
joblib.dump(f_reg, "my_model.pkl")

['my_model.pkl']

### Restore Model

In [70]:
my_model_restored = joblib.load("my_model.pkl")